In [146]:
#!/usr/bin/env python3

This script controls preprocessing of data required ore advantageous for
subsequent inversions.


In [147]:
import os
import sys
from sys import exit as error
import copy
from time import process_time
from datetime import datetime
from random import randrange
import warnings

In [148]:
import numpy

In [149]:
AEMPYX_ROOT = os.environ["AEMPYX_ROOT"]
mypath = [AEMPYX_ROOT+"/aempy/modules/", AEMPYX_ROOT+"/aempy/scripts/"]
for pth in mypath:
    if pth not in sys.path:
        sys.path.insert(0,pth)

In [150]:
from version import versionstrg
import util
import prep
import aesys

In [151]:

AEMPYX_DATA = os.environ["AEMPYX_DATA"]

In [152]:
version, _ = versionstrg()
script = "Tutorial2_PRE_process_flightline.py"
# fname = __file__  # this only works in python, not jupyter notebook
titstrng = util.print_title(version=version, fname=script, out=False)
print(titstrng+"\n\n")
Header = titstrng

AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py




Define some parameters required for the different systems.

In [153]:
# AEM_system = "genesis"
AEM_system = "aem05"

if "aem05" in AEM_system.lower():
    _, NN, _, _, _, = aesys.get_system_params(System=AEM_system)
    nD = NN[0]

if "genes" in AEM_system.lower():
    _, NN, _, _, _, = aesys.get_system_params(System=AEM_system)
    nD = NN[0]


AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]


In [154]:
OutInfo = True
OutNaN = True
OutRes = False

SingValMax = 5

In [155]:
InFileFmt = ".npz"

Filelist = "search" # "set", "read"
SearchStrng = "*FL*.npz"

OutFileFmt = ".npz"


In [156]:
AEMPYX_DATA =  AEMPYX_ROOT
InputDataDir =  AEMPYX_DATA + "/work/Limerick/raw/"
OutputDataDir =  AEMPYX_DATA + "/work/Limerick/proc/"

In [157]:
print("\n\n")
print("Data read from dir:  %s" % InputDataDir)
print("Search flightline ID string: %s " % SearchStrng)
print("Processed data  written to dir: %s " % OutputDataDir)




Data read from dir:  /home/vrath/AEMpyX//work/Limerick/raw/
Search flightline ID string: *FL*.npz 
Processed data  written to dir: /home/vrath/AEMpyX//work/Limerick/proc/ 


In [158]:
if not os.path.isdir(OutputDataDir):
    print("File: %s does not exist, but will be created" % OutputDataDir)
    os.mkdir(OutputDataDir)

In [159]:
print("Data read from dir: %s " % InputDataDir)

Data read from dir: /home/vrath/AEMpyX//work/Limerick/raw/ 


In [160]:
dat_files = util.get_data_list(how=["search", SearchStrng, InputDataDir],
                              out= True, sort=True)
ns = numpy.size(dat_files)
if ns ==0:
    error("No files corresponding to searchstring <"+SearchStrng+"> found!. Exit.")

Data files read from dir:  /home/vrath/AEMpyX//work/Limerick/raw/
Search string: *FL*.npz 
76 data files found:
['A5_rect_shale_FL5001-0.npz', 'A5_rect_shale_FL5002-0.npz', 'A5_rect_shale_FL5003-0.npz', 'A5_rect_shale_FL5004-0.npz', 'A5_rect_shale_FL5005-0.npz', 'A5_rect_shale_FL5006-0.npz', 'A5_rect_shale_FL5007-0.npz', 'A5_rect_shale_FL5008-0.npz', 'A5_rect_shale_FL5009-0.npz', 'A5_rect_shale_FL5010-0.npz', 'A5_rect_shale_FL5011-01.npz', 'A5_rect_shale_FL5012-0.npz', 'A5_rect_shale_FL5013-0.npz', 'A5_rect_shale_FL5014-0.npz', 'A5_rect_shale_FL5015-0.npz', 'A5_rect_shale_FL5016-0.npz', 'A5_rect_shale_FL5017-0.npz', 'A5_rect_shale_FL5018-0.npz', 'A5_rect_shale_FL5019-0.npz', 'A5_rect_shale_FL5020-0.npz', 'A5_rect_shale_FL5021-0.npz', 'A5_rect_shale_FL5022-0.npz', 'A5_rect_shale_FL5023-0.npz', 'A5_rect_shale_FL5024-0.npz', 'A5_rect_shale_FL5025-0.npz', 'A5_rect_shale_FL5026-0.npz', 'A5_rect_shale_FL5027-0.npz', 'A5_rect_shale_FL5028-0.npz', 'A5_rect_shale_FL5029-0.npz', 'A5_rect_shale_F

In [161]:
start = process_time()
num_sites = 0
num_files = 0
bad_files = 0

In [162]:
for filename in dat_files:
    num_files = num_files+1
    name, ext = os.path.splitext(filename)
    filein = InputDataDir + filename
    print("\n Preprocessing file " + filein)
    Data, Header, _ = aesys.read_aempy(File=filein,
                                   System=AEM_system, OutInfo=False)


    now = datetime.now()
    aesys.print_header(Header)
    Header = aesys.grow_header(Header, titstrng)


    if numpy.size(Data)<=nD:
        print("Not enough data! Not processed")
        continue

    D = Data[:, :]

    D = numpy.where(D<1.e30,D, numpy.nan)
    nN = numpy.count_nonzero(numpy.isnan(D))
    print (str(nN)+" NaN in Data Block")
    if nN >0:
        bad_files = bad_files+1
        continue

    fline = Data[:, 0]
    sizedat = numpy.shape(D)
    nvars = sizedat[1]
    last = nvars - 1
    print("Flightline Data block on input has shape: ", numpy.shape(D))

    action = "alt lowpass filter"
    print("\n Proc action: " + action)
    columns = [4, 5]
    print(" dcolumns: ", columns)
    Header = aesys.grow_header(Header, "LPF, IIR n=8")
    D, comment = prep.filter_column(D, columns, method=["butter", 4, 1.0 / 20.0])
    print(" data block now has shape: ", numpy.shape(D))

    action = " plm lowpass filter"
    print("\n Proc action: " + action)
    columns = [14, 14]
    print(" dcolumns: ", columns)
    Header = aesys.grow_header(Header, "LPF, IIR n=8")
    D, comment = prep.filter_column(D, columns, method=["butter", 4, 1.0 / 20.0])
    print(" data block now has shape: ", numpy.shape(D))

    action = "plm threshold "
    plmthresh = 3.
    threshval = plmthresh
    columns = [14, 14]
    print("\n Proc action: " + action)
    print(" columns: ", columns)
    print(" thresh = ", threshval)
    Header = aesys.grow_header(
        Header, "PLM, threshold = " + str(threshval))
    D, nanindex = prep.insert_flag(D, action, threshval, columns,
                                    System=AEM_system)

    action = "less than"
    threshval = -500.0
    columns = [6, 14]
    print("\n Proc action: " + action)
    print(" columns: ", columns)
    print(" thresh = ", threshval)
    Header = aesys.grow_header(
        Header, "DAT, threshold = " + str(threshval))
    D, nanindex = prep.insert_flag(D, action, threshval, columns,
                                   System=AEM_system)
    action = "greater than"
    threshval = 100.0
    columns = [4, 4]
    print("\n Proc action: " + action)
    print(" columns: ", columns)
    print(" thresh = ", threshval)
    Header = aesys.grow_header(
        Header, "ALT, threshold = " + str(threshval))
    D, nanindex = prep.insert_flag(D, action, threshval, columns,
                                   System=AEM_system)

    print("Info:")
    print(Header)
    print("time taken = ", process_time() - start, "s \n")

    if OutNaN:    
        OutNameStrng = name + "_proc"
        filout = OutputDataDir + OutNameStrng +"_nan"+OutFileFmt
        aesys.write_aempy(File=filout, Data=D, System=AEM_system,
                            Header=Header, OutInfo=False)
        print("Data with NaN written to File: " + filout)


    # impute = ["noise", 100.]
    action = "handle_gaps"
    impute = ["delete", 0.]
    print("Impute method:")
    print(impute)
    columns = [6, 14]
    Header = aesys.grow_header(
        Header, "GAP, method = " + impute[0])
    D = prep.handle_gaps(D, columns, Impute=impute, System=AEM_system)
    print(" data block now has shape: ", numpy.shape(D))

    if numpy.shape(D)[0] == 0:
        continue
        
        
    OutNameStrng = name + "proc_"+impute[0]+"_PLM"+str(int(plmthresh))+"s"
    filout = OutputDataDir + OutNameStrng + OutFileFmt
    aesys.write_aempy(File=filout, Data=D, System=AEM_system,
                    Header=Header, OutInfo=False)

    print("Imputed data written to File: " + filout)
    print("Info:")
    print(Header)
    print("time taken = ", process_time() - start, "s \n")

    nDfinal = numpy.shape(D)
    num_sites = num_sites + nDfinal[0]

    print("\nRunning pca ")
    columns = [6, 13]
    ncols = numpy.size(range(columns[0], columns[1] + 1))
    F = numpy.zeros(SingValMax)

    k = 0
    M = numpy.zeros(SingValMax)
    SVals = numpy.nan * numpy.ones((0, 8))
    MVals = numpy.nan * numpy.ones((0, 8))
    while k < SingValMax:

        k = k + 1
        print(" N pca: ", k)
        Data_k, U, S, V, MSE, FRO = prep.calc_svd_decomp(D, columns, k=k,
                                                  out_full=True)
        S = S / S[0]
        F[k-1] = FRO
        if OutInfo:
            print("TSVD: "+" k="+str(k)+" S(rel)="+str(S)+" FRO="+str(FRO))


        head = aesys.grow_header(
            Header,"TSVD: "+" k="+str(k)+" S(rel)="+str(S)+" FRO="+str(FRO))
        
        OutNameStrng = name + "_proc_"+impute[0]+"_k" + str(k)
        filout = OutputDataDir + OutNameStrng+ OutFileFmt
        aesys.write_aempy(File=filout, Data=Data_k,
                        System=AEM_system, Header=head, OutInfo=False)
        print("Data written to File: " + filout)
        print("Info:")
        print(head)
        print("time taken = ", process_time() - start, "s \n")

        if OutRes:
            D_res = copy.deepcopy(D)
            nd1 = NN[1]
            nd2 = NN[1]+ NN[2]
            D_res[:,nd1:nd2] = D[:,nd1:nd2]-Data_k[:,nd1:nd2]
            print("min, max = ",numpy.amin(D_res[:,nd1:nd2]), ", ",numpy.amax(D_res[:,nd1:nd2]))
            print("std = ",numpy.std(D_res[:,nd1:nd2]))
            head = aesys.grow_header(
                Header,"TSVD: "+" k="+str(k)+" Res = min "+str(numpy.amin(D_res[:,nd1:nd2]))
                                                +" / max "+str(numpy.amax(D_res[:,nd1:nd2]))
                                                +" / std "+str(numpy.std(D_res[:,nd1:nd2])))

            OutNameStrng = name + "_proc_"+inpute[0]+"_k" + str(k)+"_res"
            filout = OutputDataDir + OutNameStrng+ OutFileFmt
            aesys.write_aempy(File=filout, Data=D_res,
                            System=AEM_system, Header=head, OutInfo=False)
            print("Data written to File: " + filout)
            print("Info:")
            print(head)
            print("time taken = ", process_time() - start, "s \n")


 Preprocessing file /home/vrath/AEMpyX//work/Limerick/raw/A5_rect_shale_FL5001-0.npz
['Data', 'Header', 'System']
Current Header:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5001.0
0 NaN in Data Block
Flightline Data block on input has shape:  (1057, 17)

 Proc action: alt lowpass filter
 dcolumns:  [4, 5]
<built-in method lower of str object at 0x7f8e4cca20b0>
filter column 4, method = ['butter', 4, 0.05]
filter column 5, method = ['butter', 4, 0.05]
 data block now has shape:  (1057, 17)

 Proc action:  plm lowpass filter
 dcolumns:  [14, 14]
<built-in method lower of str object at 0x7f8e4cca20b0>
filter column 14, method = ['butter', 4, 0.05]
 data block now has shape:  (1057, 17)

 Proc action: plm threshold 
 columns:  [14, 14]
 thresh =  3.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]

 Proc action: less than
 columns:  [6, 14]
 thresh 

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5004-0_proc_delete_k5.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5004.0 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=5 S(rel)=[1.         0.44142554 0.1374952  0.10078524 0.08958813 0.07155228
 0.0471412  0.04523264] FRO=0.08696463158241974
time taken =  0.41038296600000024 s 


 Preprocessing file /home/vrath/AEMpyX//work/Limerick/raw/A5_rect_shale_FL5005-0.npz
['Data', 'Header', 'System']
Current Header:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5005.0
0 NaN in Data Bloc

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5008-0_proc_delete_k2.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5008.0 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=2 S(rel)=[1.         0.39564641 0.14789607 0.13516303 0.10577031 0.07443277
 0.05546342 0.05297253] FRO=0.2268616967368708
time taken =  2.0424510310000006 s 

 N pca:  3
(1778, 8)
TSVD:  k=3 S(rel)=[1.         0.39564641 0.14789607 0.13516303 0.10577031 0.07443277
 0.05546342 0.05297253] FRO=0.18310344751986626
Output file format is .npz
Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5008-0_proc_delete_k3.npz
Info:
AEMpy

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5011-01_proc_delete_k1.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5011.01 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=1 S(rel)=[1.         0.45047156 0.16578555 0.12351154 0.04855402 0.03546572
 0.02985816 0.02630413] FRO=0.44783047584925667
time taken =  3.645177277 s 

 N pca:  2
(2133, 8)
TSVD:  k=2 S(rel)=[1.         0.45047156 0.16578555 0.12351154 0.04855402 0.03546572
 0.02985816 0.02630413] FRO=0.19576637475662
Output file format is .npz
Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5011-01_proc_delete_k2.npz
Info:
AEMpyX Vers

Imputed data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5014-0proc_delete_PLM3s.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5014.0 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete
time taken =  5.339791847 s 


Running pca 
 N pca:  1
(1791, 8)
TSVD:  k=1 S(rel)=[1.         0.45982411 0.19581023 0.10310917 0.08018906 0.06707225
 0.05531936 0.04824718] FRO=0.46556205594074007
Output file format is .npz
Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5014-0_proc_delete_k1.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.p

...

filter column 4, method = ['butter', 4, 0.05]
filter column 5, method = ['butter', 4, 0.05]
 data block now has shape:  (1468, 17)

 Proc action:  plm lowpass filter
 dcolumns:  [14, 14]
<built-in method lower of str object at 0x7f8e4cca20b0>
filter column 14, method = ['butter', 4, 0.05]
 data block now has shape:  (1468, 17)

 Proc action: plm threshold 
 columns:  [14, 14]
 thresh =  3.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]

 Proc action: less than
 columns:  [6, 14]
 thresh =  -500.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]

 Proc action: greater than
 columns:  [4, 4]
 thresh =  100.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5056.01 | AEMpyX Version 0.99.99
Date 05/12/2024, 

Current Header:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5062.01
0 NaN in Data Block
Flightline Data block on input has shape:  (652, 17)

 Proc action: alt lowpass filter
 dcolumns:  [4, 5]
<built-in method lower of str object at 0x7f8e4cca20b0>
filter column 4, method = ['butter', 4, 0.05]
filter column 5, method = ['butter', 4, 0.05]
 data block now has shape:  (652, 17)

 Proc action:  plm lowpass filter
 dcolumns:  [14, 14]
<built-in method lower of str object at 0x7f8e4cca20b0>
filter column 14, method = ['butter', 4, 0.05]
 data block now has shape:  (652, 17)

 Proc action: plm threshold 
 columns:  [14, 14]
 thresh =  3.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]

 Proc action: less than
 columns:  [6, 14]
 thresh =  -500.0

AEM system is aem05
Forward model call: core1d.aemfwd1d_aem05(nlyr, m, alt)
Data:[17, 6, 8, 3]

 Proc acti

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL5063-0_proc_delete_k5.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5063.0 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=5 S(rel)=[1.         0.3393101  0.24887098 0.19230265 0.15403826 0.10788524
 0.09125675 0.08068453] FRO=0.14471629839461259
time taken =  27.220250571 s 


 Preprocessing file /home/vrath/AEMpyX//work/Limerick/raw/A5_rect_shale_FL5064-0.npz
['Data', 'Header', 'System']
Current Header:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 5064.0
0 NaN in Data Block
Fligh

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL515-0_proc_delete_k3.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 515.0 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=3 S(rel)=[1.         0.75325679 0.26790563 0.10405731 0.08317482 0.05692221
 0.0526084  0.03794802] FRO=0.12303335995910958
time taken =  27.927254138000002 s 

 N pca:  4
(1880, 8)
TSVD:  k=4 S(rel)=[1.         0.75325679 0.26790563 0.10405731 0.08317482 0.05692221
 0.0526084  0.03794802] FRO=0.09290565989282094
Output file format is .npz
Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL515-0_proc_delete_k4.npz
Info:
AEMpyX 

Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL519-01_proc_delete_k1.npz
Info:
AEMpyX Version 0.99.99
Date 05/12/2024, 11:10:23
Tutorial1_PRE_data.py, modified 05/12/2024, 11:10:08
Tutorial1_PRE_data.py | Flightline 519.01 | AEMpyX Version 0.99.99
Date 05/12/2024, 20:27:11
Tutorial2_PRE_process_flightline.py, modified 05/12/2024, 20:26:18
Tutorial2_PRE_process_flightline.py | LPF, IIR n=8 | LPF, IIR n=8 | PLM, threshold = 3.0 | DAT, threshold = -500.0 | ALT, threshold = 100.0 | GAP, method = delete | TSVD:  k=1 S(rel)=[1.         0.42548128 0.12889946 0.09856772 0.07892236 0.06807732
 0.06588598 0.04799978] FRO=0.4284728179395151
time taken =  29.557841663999998 s 

 N pca:  2
(1795, 8)
TSVD:  k=2 S(rel)=[1.         0.42548128 0.12889946 0.09856772 0.07892236 0.06807732
 0.06588598 0.04799978] FRO=0.18918386763703027
Output file format is .npz
Data written to File: /home/vrath/AEMpyX//work/Limerick/proc/A5_rect_shale_FL519-01_proc_delete_k2.npz
Info:
AEMpy

In [163]:
print("\nAll done!")


All done!


In [164]:
elapsed = process_time() - start
print(" Used %7.4f sec for %6i lines  - %8i sites\n" % (elapsed, num_files + 1, num_sites))

 Used 30.1038 sec for     77 lines  -   112443 sites

